In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
%load_ext quak

# Load Reading

In [3]:
data_cols = {
    "CDA03ARE1223R": "reading", # Campus 2023 Domain 1a: Meets Grade Level Std, Grade 3, All Students, Staar Reading/Ela Rate 
    # "CAMPNAME": "campus_name",
    # "DISTRICT": "district_number",
    "CAMPUS": "campus_number",
    "DISTNAME": "district_name"}



From the data dictionary:

'-' Indicates there are no students in the group.

'*' Indicates results are masked due to small numbers to protect student confidentiality.

** When only one student disability group is masked, then the second smallest student disability group is masked regardless of size.

n/a Indicates data reporting is not applicable for this group.

? Indicates that the data for this item were statistically improbable or were reported outside a reasonable range.

In [4]:
na_values = ['-', '*', '**', 'n/a', pd.NA, np.nan, '?']
# None of these are actually used in this table (nor were they mentioned on the table's
# specific page in the data dictionary. But I do see -1 and '.' so I will add those.
na_values += ['-1', '.']

In [5]:
df = pd.read_csv("../data/raw/CSTAAR_GR3.csv",na_values=na_values, usecols=data_cols.keys())

In [6]:
df = df.rename(columns=data_cols)

In [7]:
df

Widget(sql='SELECT * FROM "df"')

In [8]:
assert all(((df['reading'].max() <= 100) & (df['reading'].min() >= 0)) | df.reading.isna())

In [9]:
assert len(df) == df['campus_number'].nunique()

In [10]:
assert df['district_name'].notna().all()

# Explore

**Callouts:**

* Most of the schools' data are not reported or masked.

In [11]:
df.reading.isna().mean()

np.float64(0.4816452896948253)

**Callouts:**

* This data is almost way too normal to be real!!
* Mean district has 50% passing.
* Outlier districts have 100% and 0%.


In [12]:
plot_data = df.groupby('district_name')['reading'].mean()
px.box(plot_data)

# Load Schools

In [13]:
schools = pd.read_excel("../data/raw/School Addresses - Hiring Project.xlsx", 
                   dtype={"School Zip":"str"})

In [14]:
data_cols = {
    "School Number": "campus_number", 
    "School Name": "campus_name", 
    "Instruction Type": "instruction_type", 
    "Charter Type":	"charter_type", 
    "School City": "city", 
    "School State":	"state", 
    "School Zip": "zip_plusfour", 
    "Grade Range": "grade_range", 
    "School Enrollment as of Oct 2023": "enrollment",
    "School Status": "status"
}

In [15]:
schools = schools.rename(columns=data_cols)

In [16]:
assert all((schools['zip_plusfour'].str.len() == 5) | (schools['zip_plusfour'].str.len() == 10))

In [17]:
schools['zip'] = schools['zip_plusfour'].str.slice(0, 5)

# Merge Schools -> Reading

In [18]:
df = df.merge(schools, on='campus_number', how='left', validate="1:1", indicator=True)

**Callouts**:

* A few dozen schools are not in the school address master list.
* The districts are "El Paso", "Fort Worth", "University of Texas", so obviously some are still in TX.

In [19]:
print(df['_merge'].value_counts())
print(df[df['_merge']=='left_only']['district_name'].unique())

_merge
both          8942
left_only      102
right_only       0
Name: count, dtype: int64
['DIBOLL ISD' 'PRIORITY CHARTER SCHOOLS' 'ORENDA CHARTER SCHOOL'
 'TEMPLE ISD' 'HARLANDALE ISD' 'EDGEWOOD ISD' 'SAN ANTONIO ISD'
 'SOUTH SAN ANTONIO ISD' 'BRAZOSPORT ISD' 'MCLEOD ISD' 'ALLEN ISD'
 'LUMIN EDUCATION' 'CEDAR HILL ISD' 'IRVING ISD' 'EL PASO ISD'
 'YSLETA ISD' 'TORNILLO ISD' 'MARLIN ISD' 'FORT BEND ISD' 'PAMPA ISD'
 'ALIEF ISD' 'HIDALGO ISD' 'WESLACO ISD' 'VALLEY VIEW ISD' 'LEVELLAND ISD'
 'FT HANCOCK ISD' 'PERRIN-WHITT CISD' 'PORT NECHES-GROVES ISD'
 'KARNES CITY ISD' 'NORTH LAMAR ISD' 'PALACIOS ISD' 'VAN VLECK ISD'
 'WACO CHARTER SCHOOL' 'WACO ISD' 'WEST ISD' 'BRUCEVILLE-EDDY ISD'
 'TEXAS SERENITY ACADEMY' 'DUMAS ISD' 'NACOGDOCHES ISD' 'DAWSON ISD'
 'FROST ISD' 'CORPUS CHRISTI ISD' 'ROBSTOWN ISD' 'PECOS-BARSTOW-TOYAH ISD'
 'WEST RUSK COUNTY CONSOLIDATED ISD' 'BROADDUS ISD' 'JOAQUIN ISD'
 'HIGH POINT ACADEMY' 'BIRDVILLE ISD' 'FORT WORTH ISD' 'AZLE ISD'
 'TEXAS COLLEGE PREPARATORY ACAD

In [20]:
df

Widget(sql='SELECT * FROM "df"')

# Explore

**Callouts**:

* Instruction type is pretty imbalanced.
* We don't have reading data for any DAEP and JJAEP schools and almost all ALT schools.

In [21]:
print(df['instruction_type'].value_counts(dropna=False))
print(df.assign(readingisna = df['reading'].isna()).groupby('instruction_type', dropna=False)['readingisna'].mean())
px.strip(df, x='instruction_type', y='reading')

instruction_type
REGULAR INSTRUCTIONAL        8205
ALTERNATIVE INSTRUCTIONAL     454
JJAEP INSTRUCTIONAL           143
DAEP INSTRUCTIONAL            140
NaN                           102
Name: count, dtype: int64
instruction_type
ALTERNATIVE INSTRUCTIONAL    0.958150
DAEP INSTRUCTIONAL           1.000000
JJAEP INSTRUCTIONAL          1.000000
REGULAR INSTRUCTIONAL        0.437172
NaN                          0.500000
Name: readingisna, dtype: float64


**Callouts**:

* Charter type is also very unbalanced.
* I'm guessing most schools are not charters.
* College charters are more likely to underreport.

In [22]:
print(df['charter_type'].value_counts(dropna=False))
print(df.assign(readingisna = df['reading'].isna()).groupby('charter_type',dropna=False)['readingisna'].mean())
px.strip(df, x='charter_type', y='reading')

charter_type
NaN                           8037
OPEN ENROLLMENT CHARTER        868
CAMPUS CHARTER                 114
COLLEGE/UNIVERSITY CHARTER      25
Name: count, dtype: int64
charter_type
CAMPUS CHARTER                0.491228
COLLEGE/UNIVERSITY CHARTER    0.760000
OPEN ENROLLMENT CHARTER       0.497696
NaN                           0.478910
Name: readingisna, dtype: float64


**Callouts:**

* Could be interesting to show city distribution or select your city within distribution.

In [23]:
plot_data = df.assign(order = df.groupby('city')['reading'].transform('mean')).sort_values('order')
px.strip(plot_data, x='city', y='reading')

**Callouts**:

* Could be interesting to show distribution of zips or select your zip.

In [24]:
plot_data = df.assign(order = df.groupby('zip')['reading'].transform('mean')).sort_values('order')
px.strip(plot_data, x='zip', y='reading')

**Callouts**:

* There is ONE school here that is in Nevada. We should drop that row.

In [25]:
mask = (df['state'] == 'TX') | (df['_merge'] == 'left-only')
print("{} non-TX rows".format(len(mask) - mask.sum()))

103 non-TX rows


**Callouts**:

* Okay there are a LOT of combinations of grades.
* I think I will simplify this. I can either one-hot code or group into broader categories.
* What I should definitely do is code it as an interval (excluding EE for now) and go from there.
* Maybe need to filter this data on grade level because lots of these don't have 3rd graders.

In [26]:
px.strip(df, x='grade_range', y='reading')

**Callouts:**

* There is possibly no trend vs enrollment.

In [27]:
px.scatter(df, x='enrollment', y='reading', log_x=True)